In [4]:
import glob
import re
import pandas as pd
import numpy as np

# --- 1. Read & normalize all your Excel sheets into price series ----

def extract_price_sheet(path, sheet_name=None):
    """
    Reads one sheet of an Excel file, finds the date col + price col,
    returns a Series indexed by date.
    """
    df = pd.read_excel(path, sheet_name=sheet_name, engine='openpyxl')
    
    # find a column whose name contains 'date'
    date_col = next(col for col in df.columns 
                    if re.search(r'date', col, re.IGNORECASE))
    # find a price-like column: NAV, Close, Price, Net, etc.
    price_col = next(col for col in df.columns 
                     if re.search(r'\b(nav|close|price|net)\b', col, re.IGNORECASE))
    
    s = df[[date_col, price_col]].dropna()
    s = s.rename(columns={date_col: 'Date', price_col: 'Price'})
    s['Date'] = pd.to_datetime(s['Date'])
    s = s.set_index('Date').sort_index()
    return s['Price']

def load_all_prices(pattern="data/*.xlsx"):
    """
    Loads every sheet from every Excel file matching the glob pattern,
    and returns a DataFrame of all price series side by side.
    """
    price_frames = []
    for fn in glob.glob(pattern):
        # try default sheet plus any others
        xls = pd.ExcelFile(fn, engine='openpyxl')
        for sh in xls.sheet_names:
            try:
                ser = extract_price_sheet(fn, sheet_name=sh)
            except StopIteration:
                continue  # skip sheets without the right headers
            # give it a name based on filename and sheet
            name = f"{fn.split('/')[-1].replace('.xlsx','')}"
            if len(xls.sheet_names) > 1:
                name += f"__{sh}"
            ser.name = name
            price_frames.append(ser)
    # join all on their dates
    prices = pd.concat(price_frames, axis=1)
    return prices

# load
prices = load_all_prices("/Users/nachogutierrezdelaroza/Documents/2_PP/WM/Fund returns/Developed ex_UK_USA/fidelity_europe.xlsx")

# --- 2. Compute returns (daily or weekly) ---

# for daily:
rets = prices.pct_change().dropna()

# OR for weekly (e.g. Friday closes):
# rets = prices.resample('W-FRI').last().pct_change().dropna()

# --- 3. Annualization factors & risk-free rate ---
trading_days = 252
rf_annual = 0.01 # This is an example; replace with your risk-free rate
rf_daily = (1 + rf_annual)**(1/trading_days) - 1

# --- 4. Basic per-series stats ---
mean_daily = rets.mean()
vol_daily = rets.std()
var_daily = rets.var()

mean_ann = mean_daily * trading_days
vol_ann = vol_daily * np.sqrt(trading_days)
var_ann = var_daily * trading_days

# --- 5. Sharpe Ratio ---
excess = rets.sub(rf_daily)
sharpe = excess.mean() / vol_daily * np.sqrt(trading_days)

# --- 6. Sortino Ratio ---
def sortino(r, rf=rf_daily, period=252):
    downside = r[r < rf]
    dd = np.sqrt((downside**2).mean()) * np.sqrt(period)
    er = (r.mean() - rf) * period
    return er / dd

sortino = rets.apply(sortino)

# --- 7. Information Ratio ---
# pick one column as benchmark, or load a separate series
benchmark = rets.iloc[:, 0]   # e.g. first series, or replace with your index
active = rets.sub(benchmark, axis=0)
ir = active.mean() / active.std() * np.sqrt(trading_days)

# --- 8. Correlation & Covariance ---
corr = rets.corr()
cov_ann = rets.cov() * trading_days

# --- 9. Summarize ---
summary = pd.DataFrame({
    'Mean Ann.': mean_ann,
    'Vol. Ann.': vol_ann,
    'Var. Ann.': var_ann,
    'Sharpe': sharpe,
    'Sortino': sortino,
    'Info Ratio': ir
}).round(4)

print("=== Performance Summary ===")
print(summary)
print("\n=== Correlation Matrix ===")
print(corr.round(3))
print("\n=== Annualized Covariance Matrix ===")
print(cov_ann.round(6))


ValueError: No objects to concatenate